# 데이터 **로딩**

In [ ]:
import pandas as pd
import numpy as np

raw_d = pd.read_csv('rawdata.csv')
cont_d = pd.read_csv('df_contents.csv') 
rat_sent = pd.read_csv('df_ratings_senti.csv')
# rat_np = pd.read_csv('df_ratings_NumPost.csv')
# movies = pd.read_csv('movies.csv')
# ratings = pd.read_csv('ratings.csv')
rat_sent = rat_sent.drop(['index'], axis=1)


In [ ]:
cont_d

In [ ]:
rat_sent

## **행렬분해잠재요인 협업 필터링**

In [ ]:
rat_sent.shape

(15413, 120)

In [11]:
rat_sent.columns[1]

'갤러리아트카페'

In [ ]:
new_data = {rat_sent.columns[0]:'0.01'}
a = rat_sent.append(new_data, ignore_index=True)
a

In [17]:
##모델 합치기 input 모델링###


def app_sent(rat_senti, senti):
  min = 1
  for i in range(len(rat_sent[0])):
    for j in range(len(rat_sent[1])):
      if rat_semti.iloc[i, j] == np.nan:
        continue
      else:
        mse = (senti-rat_senti.iloc[i, j])**2

      if min > mse:
        min = mse
        row = i
        col = j

  return row, rat_senti.columns[col]

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    # 두개의 분해된 행렬 P와 Q.T의 내적 곱으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P, Q.T)
    
    # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출하여 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
      
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse

def matrix_factorization(R, K, steps=200, learning_rate=0.01, r_lambda = 0.01):
    num_users, num_items = R.shape
    # P와 Q 매트릭스의 크기를 지정하고 정규분포를 가진 랜덤한 값으로 입력합니다. 
    np.random.seed(1)
    P = np.random.normal(scale=1./K, size=(num_users, K))
    Q = np.random.normal(scale=1./K, size=(num_items, K))

    break_count = 0
       
    # R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트 객체에 저장. 
    non_zeros = [ (i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0 ]
   
    # SGD기법으로 P와 Q 매트릭스를 계속 업데이트. 
    for step in range(steps):
        for i, j, r in non_zeros:
            # 실제 값과 예측 값의 차이인 오류 값 구함
            eij = r - np.dot(P[i, :], Q[j, :].T)
            # Regularization을 반영한 SGD 업데이트 공식 적용
            P[i,:] = P[i,:] + learning_rate*(eij * Q[j, :] - r_lambda*P[i,:])
            Q[j,:] = Q[j,:] + learning_rate*(eij * P[i, :] - r_lambda*Q[j,:])
       
        rmse = get_rmse(R, P, Q, non_zeros)
        if (step % 10) == 0 :
            print("### iteration step : ", step," rmse : ", rmse)
            
    return P, Q

import pandas as pd
import numpy as np



# movies = pd.read_csv('df_contents.csv')  ## cont_d ##
# ratings = pd.read_csv('df_ratings_senti.csv') ## rat_sent ##
# ratings = rat_sent[['userId', 'movieId', 'rating']]
# ratings_matrix = ratings.pivot_table('rating', index='userId', columns='movieId')

# # title 컬럼을 얻기 이해 movies 와 조인 수행
# rating_movies = pd.merge(ratings, movies, on='movieId')

# # columns='title' 로 title 컬럼으로 pivot 수행. 
# ratings_matrix = rating_movies.pivot_table('rating', index='userId', columns='title')

ratings_matrix = rat_sent
ratings_matrix = ratings_matrix.fillna(0)

P, Q = matrix_factorization(ratings_matrix.values, K=50, steps=200, learning_rate=0.01, r_lambda = 0.01)
pred_matrix = np.dot(P, Q.T)


ratings_pred_matrix = pd.DataFrame(data=pred_matrix, index= ratings_matrix.index,
                                   columns = ratings_matrix.columns)

ratings_pred_matrix.head(3)



def get_ungone_cafe(ratings_matrix, userId):
    # userId로 입력받은 사용자의 모든 영화정보 추출하여 Series로 반환함. 
    # 반환된 user_rating 은 영화명(title)을 index로 가지는 Series 객체임. 
    user_rating = ratings_matrix.loc[userId,:]
    
    # user_rating이 0보다 크면 기존에 관람한 영화임. 대상 index를 추출하여 list 객체로 만듬
    already_gone = user_rating[ user_rating > 0].index.tolist()
    
    # 모든 영화명을 list 객체로 만듬. 
    cafe_list = ratings_matrix.columns.tolist()
    
    # list comprehension으로 already_seen에 해당하는 movie는 movies_list에서 제외함. 
    ungone_list = [ cafe for cafe in cafe_list if cafe not in already_gone]
    
    return ungone_list

def recomm_cafe_by_userid(pred_df, userId, ungone_list, top_n=10):
    # 예측 평점 DataFrame에서 사용자id index와 ungone_list로 들어온 영화명 컬럼을 추출하여
    # 가장 예측 평점이 높은 순으로 정렬함. 
    recomm_cafe = pred_df.loc[userId, ungone_list].sort_values(ascending=False)[:top_n]
    return recomm_cafe

#==============================================================ㅠ
# 웹으로부터 입력
sentiment = input()
row, col_name = app_sent(rat_sent, sentiment)

# 사용자가 관람하지 않는 영화명 추출   
ungone_list = get_ungone_cafe(ratings_matrix, row)

# 아이템 기반의 인접 이웃 협업 필터링으로 영화 추천 ###인풋값
recomm_cafe = recomm_cafe_by_userid(ratings_pred_matrix, row, ungone_list, top_n=10)

# 평점 데이타를 DataFrame으로 생성. ###인풋값
recomm_cafe = pd.DataFrame(data=recomm_cafe.values,index=recomm_cafe.index,columns=['pred_score'])
recomm_cafe


# dfdf = pd.read_pickle('cafe_df.pkl')


### iteration step :  0  rmse :  0.8475184394048498
### iteration step :  10  rmse :  0.7889527932481144
### iteration step :  20  rmse :  0.6692337233650227
### iteration step :  30  rmse :  0.4917260703384629
### iteration step :  40  rmse :  0.3419311780554512
### iteration step :  50  rmse :  0.24134578760595546
### iteration step :  60  rmse :  0.17696279957865363
### iteration step :  70  rmse :  0.13341384068737677
### iteration step :  80  rmse :  0.10352409947192862
### iteration step :  90  rmse :  0.08354118460638313
### iteration step :  100  rmse :  0.06992683282862379
### iteration step :  110  rmse :  0.06032184564688769
### iteration step :  120  rmse :  0.053554172329201365
### iteration step :  130  rmse :  0.0488264344320518
### iteration step :  140  rmse :  0.04549258472279752
### iteration step :  150  rmse :  0.043076174034252804
### iteration step :  160  rmse :  0.04125199920400848
### iteration step :  170  rmse :  0.0398058586757243
### iteration step :  180 

,pred_score
서면코지타운,0.365598
서면투썸플레이스,0.338577
서면분커,0.328764
라데세르,0.326207
로드209,0.315835
카페안에사주와타로,0.283729
서면듀랩,0.274655
인레스트카페,0.269082
서면러브썸,0.223658
서면모모치,0.201757


# **컨텐츠 기반 **

메뉴 이용

In [18]:
df = pd.read_csv('df_contents.csv') #데이터 전처리
# df
result = []
column = ["Unnamed: 0","이름", "카테고리", "url","위도_x","경도_x","검색어","가게해시태그" ,"메뉴", "like_x", "sentiment", "happy", "sad", "angry", "surprised", "count"]
for i in range(len(df)):
    place_info = []
#     print(df["카테고리"][i])
    place_info.append(df["Unnamed: 0"][i])
    place_info.append(df["이름"][i])
    place_info.append(df["카테고리"][i].replace(" ",""))
    place_info.append(df["url"][i])
    place_info.append(df["위도_x"][i])
    place_info.append(df["경도_x"][i])
    place_info.append(df["검색어"][i])
#     print(df["가게해시태그"][i][1:-1].replace("'","").replace("#","").split(","))
    place_info.append(df["가게해시태그"][i][1:-1].replace("'","").replace("#","").split(","))
    place_info.append(df["메뉴"][i][1:-1].replace("'","").split(","))
    place_info.append(df["like_x"][i])
    place_info.append(df["sentiment"][i])
    place_info.append(df["happy"][i])
    place_info.append(df["sad"][i])
    place_info.append(df["angry"][i])
    place_info.append(df["surprised"][i])
    place_info.append(df["count"][i])

#     print(place_info)
    result.append(place_info)
cont_d = pd.DataFrame(result, columns = column)
# df.to_csv('df_contents.csv', columns = column, header = True)
# categorys = []

# cont_d = pd.read_csv('df_contents.csv')
cont_d.head(10) 

import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

# cafe = pd.read_csv('df_contents.csv')
# print(cafe.shape)
# print(cafe.head(1))

cafe = cont_d
cafe_df = cafe[['Unnamed: 0', '이름', '카테고리', '메뉴', '가게해시태그', 'sentiment', 'like_x', 'count']]
# movies_df = movies[['id','title', 'genres', 'vote_average', 'vote_count',
#                  'popularity', 'keywords', 'overview']]

######dictionary -> list #########
# pd.set_option('max_colwidth', 100)
# movies_df[['genres','keywords']][:1]

# from ast import literal_eval

# movies_df['genres'] = movies_df['genres'].apply(literal_eval)
# movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)


# movies_df['genres'] = movies_df['genres'].apply(lambda x : [ y['name'] for y in x])
# movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [ y['name'] for y in x])
# movies_df[['genres', 'keywords']][:1]




from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환. 
cafe_df['menu_literal'] = cafe_df['메뉴'].apply(lambda x : (' ').join(x))
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
menu_mat = count_vect.fit_transform(cafe_df['menu_literal'])
cafe_df['menu_mat'] = count_vect.fit_transform(cafe_df['menu_literal'])

print(menu_mat.shape)
print('\n')



from sklearn.metrics.pairwise import cosine_similarity

menu_sim = cosine_similarity(menu_mat, menu_mat)
print(menu_sim.shape)
print('\n')

print(menu_sim[:2])
print('\n')


menu_sim_sorted_ind = menu_sim.argsort()[:, ::-1]
print(menu_sim_sorted_ind[:1])
print('\n')



def find_sim_cafe(df, sorted_ind, title_name, top_n=10):
    
    # 인자로 입력된 movies_df DataFrame에서 'title' 컬럼이 입력된 title_name 값인 DataFrame추출
    name_cafe = df[df['이름'] == title_name]
    
    # title_named을 가진 DataFrame의 index 객체를 ndarray로 반환하고 
    # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n 개의 index 추출
    title_index = name_cafe.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]
    
    # 추출된 top_n index들 출력. top_n index는 2차원 데이터 임. 
    #dataframe에서 index로 사용하기 위해서 1차원 array로 변경
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]



similar_cafe = find_sim_cafe(cafe_df, menu_sim_sorted_ind, '넉아웃',10)
similar_cafe[['이름', 'sentiment']]

################################################################################################
#########################################투표 보정식#############################################
###########################################################################################

cafe_df[['이름','sentiment','count']].sort_values('sentiment', ascending=False)[:10]


C = cafe_df['sentiment'].mean()
m = cafe_df['count'].quantile(0.6)
print('C:',round(C,3), 'm:',round(m,3))
print('\n')



percentile = 0.6
m = cafe_df['count'].quantile(percentile)
C = cafe_df['sentiment'].mean()

def weighted_vote_average(record):
    v = record['count']
    R = record['sentiment']
    
    return ( (v/(v+m)) * R ) + ( (m/(m+v)) * C )   

cafe_df['weighted_vote'] = cafe_df.apply(weighted_vote_average, axis=1)



cafe_df[['이름','sentiment','weighted_vote','count']].sort_values('weighted_vote', ascending=False)[:10]


def find_sim_cafe(df, sorted_ind, title_name, top_n=10):
    title_cafe = df[df['이름'] == title_name]
    title_index = title_cafe.index.values
    
    # top_n의 2배에 해당하는 쟝르 유사성이 높은 index 추출 
    similar_indexes = sorted_ind[title_index, :(top_n*2)]
    similar_indexes = similar_indexes.reshape(-1)
# 기준 영화 index는 제외
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    # top_n의 2배에 해당하는 후보군에서 weighted_vote 높은 순으로 top_n 만큼 추출 
    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]

print('\n')
similar_cafe = find_sim_cafe(cafe_df, menu_sim_sorted_ind, col_name,10)
similar_cafe[['이름', 'sentiment', 'weighted_vote']]
# print(menu_mat)

(118, 958)


(118, 118)


[[1.         0.12830006 0.10976426 0.22222222 0.11111111 0.11111111
  0.11111111 0.         0.         0.         0.         0.
  0.         0.         0.         0.22222222 0.         0.02579411
  0.         0.0860663  0.12598816 0.11111111 0.         0.
  0.11111111 0.11111111 0.         0.         0.         0.
  0.         0.11111111 0.         0.         0.         0.
  0.10050378 0.33333333 0.11111111 0.11111111 0.         0.11111111
  0.11111111 0.37796447 0.         0.22222222 0.36980013 0.33333333
  0.09245003 0.12598816 0.         0.09245003 0.         0.12598816
  0.11111111 0.         0.11111111 0.         0.22222222 0.10050378
  0.11111111 0.13333333 0.11111111 0.11111111 0.18490007 0.
  0.22222222 0.05337605 0.11111111 0.11111111 0.11111111 0.11111111
  0.11111111 0.17978663 0.11111111 0.09245003 0.11111111 0.
  0.22222222 0.         0.19245009 0.11111111 0.11111111 0.22222222
  0.         0.11111111 0.         0.         0.         0.11111111
  

NameError: ignored

카테고리 이용

In [ ]:

import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

# cafe = pd.read_csv('df_contents.csv')
# print(cafe.shape)
# print(cafe.head(1))




# cafe = cont_d
# cafe_df = cafe[['Unnamed: 0', '이름', '카테고리', '메뉴', '가게해시태그', 'sentiment', 'like_x', 'count']]



# movies_df = movies[['id','title', 'genres', 'vote_average', 'vote_count',
#                  'popularity', 'keywords', 'overview']]

######dictionary -> list #########
# pd.set_option('max_colwidth', 100)
# movies_df[['genres','keywords']][:1]

# from ast import literal_eval

# movies_df['genres'] = movies_df['genres'].apply(literal_eval)
# movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)


# movies_df['genres'] = movies_df['genres'].apply(lambda x : [ y['name'] for y in x])
# movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [ y['name'] for y in x])
# movies_df[['genres', 'keywords']][:1]



####벡터화#####
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
# CountVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환. 
cafe_df['menu1_literal'] = cafe_df['카테고리'].apply(lambda x : ('').join(x))
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
menu1_mat = count_vect.fit_transform(cafe_df['menu1_literal'])
cafe_df['menu1_mat'] = count_vect.fit_transform(cafe_df['menu1_literal'])
print(menu1_mat.shape)
print('\n')



from sklearn.metrics.pairwise import cosine_similarity
menu1_sim = cosine_similarity(menu1_mat, menu1_mat)
print(menu1_sim.shape)
print('\n')
print(menu1_sim[:2])
print('\n')

###내림차순####
menu1_sim_sorted_ind = menu1_sim.argsort()[:, ::-1]
print(menu1_sim_sorted_ind[:1])
print('\n')




def find_sim_cafe(df, sorted_ind, title_name, top_n=10):
    # 인자로 입력된 movies_df DataFrame에서 'title' 컬럼이 입력된 title_name 값인 DataFrame추출
    name_cafe = df[df['이름'] == title_name]
    # title_named을 가진 DataFrame의 index 객체를 ndarray로 반환하고 
    # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n 개의 index 추출
    title_index = name_cafe.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]
    # 추출된 top_n index들 출력. top_n index는 2차원 데이터 임. 
    #dataframe에서 index로 사용하기 위해서 1차원 array로 변경
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    return df.iloc[similar_indexes]



similar_cafe = find_sim_cafe(cafe_df, menu1_sim_sorted_ind, '넉아웃',10)
print('\n)')
print(similar_cafe[['이름', 'sentiment']])
print('\n')

###################################################################################################################
#################################카페 평점 필터링 최종 추천##########################################################
##########################################################################################################
cafe_df[['이름','sentiment','count']].sort_values('sentiment', ascending=False)[:10]


C = cafe_df['sentiment'].mean()
m = cafe_df['count'].quantile(0.6)
print('C:',round(C,3), 'm:',round(m,3))
print('\n')



percentile = 0.6
m = cafe_df['count'].quantile(percentile)
C = cafe_df['sentiment'].mean()

def weighted_vote_average(record):
    v = record['count']
    R = record['sentiment']
    return ( (v/(v+m)) * R ) + ( (m/(m+v)) * C )   

cafe_df['weighted_vote'] = cafe_df.apply(weighted_vote_average, axis=1)



cafe_df[['이름','sentiment','weighted_vote','count']].sort_values('weighted_vote', ascending=False)[:10]


def find_sim_cafe(df, sorted_ind, title_name, top_n=118):
    title_cafe = df[df['이름'] == title_name]
    title_index = title_cafe.index.values
    
    # top_n의 2배에 해당하는 쟝르 유사성이 높은 index 추출 
    similar_indexes = sorted_ind[title_index, :(top_n*2)]
    similar_indexes = similar_indexes.reshape(-1)
    # 기준 영화 index는 제외
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    # top_n의 2배에 해당하는 후보군에서 weighted_vote 높은 순으로 top_n 만큼 추출 
    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]

# def find_sim_cafe(df, sorted_ind, title_name, top_n=10):
#     # 인자로 입력된 movies_df DataFrame에서 'title' 컬럼이 입력된 title_name 값인 DataFrame추출
#     name_cafe = df[df['이름'] == title_name]
#     # title_named을 가진 DataFrame의 index 객체를 ndarray로 반환하고 
#     # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n 개의 index 추출
#     title_index = name_cafe.index.values
#     similar_indexes = sorted_ind[title_index, :(top_n)]
#     # 추출된 top_n index들 출력. top_n index는 2차원 데이터 임. 
#     #dataframe에서 index로 사용하기 위해서 1차원 array로 변경
#     print(similar_indexes)
#     similar_indexes = similar_indexes.reshape(-1)
#     return df.iloc[similar_indexes]



similar_cafe = find_sim_cafe(cafe_df, menu1_sim_sorted_ind, '넉아웃')
similar_cafe[['이름', 'sentiment', 'weighted_vote']]




(118, 44)


(118, 118)


[[1.         0.77459667 0.77459667 1.         0.65465367 0.65465367
  1.         0.77459667 0.77459667 0.65465367 0.77459667 0.57735027
  0.65465367 0.77459667 0.77459667 0.77459667 0.65465367 0.77459667
  0.65465367 0.65465367 1.         1.         0.65465367 0.65465367
  1.         1.         0.77459667 0.         0.65465367 0.33333333
  0.77459667 0.77459667 0.77459667 1.         0.77459667 0.77459667
  1.         0.65465367 1.         0.65465367 1.         0.77459667
  0.77459667 1.         0.65465367 0.77459667 1.         1.
  0.77459667 1.         0.65465367 0.65465367 1.         0.65465367
  1.         0.65465367 1.         0.77459667 1.         1.
  1.         0.65465367 1.         0.65465367 1.         1.
  1.         0.65465367 1.         0.77459667 1.         0.77459667
  0.65465367 1.         1.         1.         0.65465367 0.77459667
  1.         1.         0.77459667 1.         1.         0.65465367
  0.77459667 1.         0.65465367 1.         1

,이름,sentiment,weighted_vote
31,더토닥,0.952295,0.920346
18,북카페심심푸리 서면본점,0.939999,0.883374
104,녹아들다녹카페,0.908175,0.875337
15,비밀정원,0.881301,0.869452
92,모던블루커피,0.890453,0.863870
...,...,...,...
0,블랙업커피 서면점,0.683073,0.731174
20,랑데자뷰 서면본점,0.707156,0.725405
43,카페가지가지 서면점,0.697316,0.721547
34,큐식이네,0.665893,0.720034


# **마무리**

In [ ]:
df = similar_cafe[['이름', 'weighted_vote']]

df2 = recomm_cafe.reset_index()
df2.columns = ['이름', 'pred_score']
df2


m=1, n=1
df3 = pd.merge(df, df2, how='left', on='이름')
df3['final'] = df3['weighted_vote']*m + df3['pred_score']*n
df3.argsort()

# **2nd track 추천**

행렬 추천

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    # 두개의 분해된 행렬 P와 Q.T의 내적 곱으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P, Q.T)
    
    # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출하여 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
      
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse

def matrix_factorization(R, K, steps=200, learning_rate=0.01, r_lambda = 0.01):
    num_users, num_items = R.shape
    # P와 Q 매트릭스의 크기를 지정하고 정규분포를 가진 랜덤한 값으로 입력합니다. 
    np.random.seed(1)
    P = np.random.normal(scale=1./K, size=(num_users, K))
    Q = np.random.normal(scale=1./K, size=(num_items, K))

    break_count = 0
       
    # R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트 객체에 저장. 
    non_zeros = [ (i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0 ]
   
    # SGD기법으로 P와 Q 매트릭스를 계속 업데이트. 
    for step in range(steps):
        for i, j, r in non_zeros:
            # 실제 값과 예측 값의 차이인 오류 값 구함
            eij = r - np.dot(P[i, :], Q[j, :].T)
            # Regularization을 반영한 SGD 업데이트 공식 적용
            P[i,:] = P[i,:] + learning_rate*(eij * Q[j, :] - r_lambda*P[i,:])
            Q[j,:] = Q[j,:] + learning_rate*(eij * P[i, :] - r_lambda*Q[j,:])
       
        rmse = get_rmse(R, P, Q, non_zeros)
        if (step % 10) == 0 :
            print("### iteration step : ", step," rmse : ", rmse)
            
    return P, Q

import pandas as pd
import numpy as np



# movies = pd.read_csv('df_contents.csv')  ## cont_d ##
# ratings = pd.read_csv('df_ratings_senti.csv') ## rat_sent ##
# ratings = rat_sent[['userId', 'movieId', 'rating']]
# ratings_matrix = ratings.pivot_table('rating', index='userId', columns='movieId')

# # title 컬럼을 얻기 이해 movies 와 조인 수행
# rating_movies = pd.merge(ratings, movies, on='movieId')

# # columns='title' 로 title 컬럼으로 pivot 수행. 
# ratings_matrix = rating_movies.pivot_table('rating', index='userId', columns='title')

ratings_matrix = rat_sent
ratings_matrix = ratings_matrix.fillna(0)

P, Q = matrix_factorization(ratings_matrix.values, K=50, steps=200, learning_rate=0.01, r_lambda = 0.01)
pred_matrix = np.dot(P, Q.T)


ratings_pred_matrix = pd.DataFrame(data=pred_matrix, index= ratings_matrix.index,
                                   columns = ratings_matrix.columns)

ratings_pred_matrix.head(3)



def get_ungone_cafe(ratings_matrix, userId):
    # userId로 입력받은 사용자의 모든 영화정보 추출하여 Series로 반환함. 
    # 반환된 user_rating 은 영화명(title)을 index로 가지는 Series 객체임. 
    user_rating = ratings_matrix.loc[userId,:]
    
    # user_rating이 0보다 크면 기존에 관람한 영화임. 대상 index를 추출하여 list 객체로 만듬
    already_gone = user_rating[ user_rating > 0].index.tolist()
    
    # 모든 영화명을 list 객체로 만듬. 
    cafe_list = ratings_matrix.columns.tolist()
    
    # list comprehension으로 already_seen에 해당하는 movie는 movies_list에서 제외함. 
    ungone_list = [ cafe for cafe in cafe_list if cafe not in already_gone]
    
    return ungone_list

def recomm_cafe_by_userid(pred_df, userId, ungone_list, top_n=10):
    # 예측 평점 DataFrame에서 사용자id index와 ungone_list로 들어온 영화명 컬럼을 추출하여
    # 가장 예측 평점이 높은 순으로 정렬함. 
    recomm_cafe = pred_df.loc[userId, ungone_list].sort_values(ascending=False)[:top_n]
    return recomm_cafe

#==============================================================
# 웹으로부터 입력
sentiment = input()
row, col_name = app_sent(rat_sent, sentiment)

# 사용자가 관람하지 않는 영화명 추출   
ungone_list = get_ungone_cafe(ratings_matrix, row)

# 아이템 기반의 인접 이웃 협업 필터링으로 영화 추천 ###인풋값
recomm_cafe = recomm_cafe_by_userid(ratings_pred_matrix, row, ungone_list, top_n=10)

# 평점 데이타를 DataFrame으로 생성. ###인풋값
recomm_cafe = pd.DataFrame(data=recomm_cafe.values,index=recomm_cafe.index,columns=['pred_score'])
recomm_cafe


# dfdf = pd.read_pickle('cafe_df.pkl')


컨텐츠 추천

In [ ]:
df = pd.read_csv('df_contents.csv') #데이터 전처리
# df
result = []
column = ["Unnamed: 0","이름", "카테고리", "url","위도_x","경도_x","검색어","가게해시태그" ,"메뉴", "like_x", "sentiment", "happy", "sad", "angry", "surprised", "count"]
for i in range(len(df)):
    place_info = []
#     print(df["카테고리"][i])
    place_info.append(df["Unnamed: 0"][i])
    place_info.append(df["이름"][i])
    place_info.append(df["카테고리"][i].replace(" ",""))
    place_info.append(df["url"][i])
    place_info.append(df["위도_x"][i])
    place_info.append(df["경도_x"][i])
    place_info.append(df["검색어"][i])
#     print(df["가게해시태그"][i][1:-1].replace("'","").replace("#","").split(","))
    place_info.append(df["가게해시태그"][i][1:-1].replace("'","").replace("#","").split(","))
    place_info.append(df["메뉴"][i][1:-1].replace("'","").split(","))
    place_info.append(df["like_x"][i])
    place_info.append(df["sentiment"][i])
    place_info.append(df["happy"][i])
    place_info.append(df["sad"][i])
    place_info.append(df["angry"][i])
    place_info.append(df["surprised"][i])
    place_info.append(df["count"][i])

#     print(place_info)
    result.append(place_info)
cont_d = pd.DataFrame(result, columns = column)
# df.to_csv('df_contents.csv', columns = column, header = True)
# categorys = []

# cont_d = pd.read_csv('df_contents.csv')
cont_d.head(10) 

import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

# cafe = pd.read_csv('df_contents.csv')
# print(cafe.shape)
# print(cafe.head(1))

cafe = cont_d
cafe_df = cafe[['Unnamed: 0', '이름', '카테고리', '메뉴', '가게해시태그', 'sentiment', 'like_x', 'count']]
# movies_df = movies[['id','title', 'genres', 'vote_average', 'vote_count',
#                  'popularity', 'keywords', 'overview']]

######dictionary -> list #########
# pd.set_option('max_colwidth', 100)
# movies_df[['genres','keywords']][:1]

# from ast import literal_eval

# movies_df['genres'] = movies_df['genres'].apply(literal_eval)
# movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)


# movies_df['genres'] = movies_df['genres'].apply(lambda x : [ y['name'] for y in x])
# movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [ y['name'] for y in x])
# movies_df[['genres', 'keywords']][:1]




from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환. 
cafe_df['menu_literal'] = cafe_df['메뉴'].apply(lambda x : (' ').join(x))
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
menu_mat = count_vect.fit_transform(cafe_df['menu_literal'])
cafe_df['menu_mat'] = count_vect.fit_transform(cafe_df['menu_literal'])

print(menu_mat.shape)
print('\n')



from sklearn.metrics.pairwise import cosine_similarity

menu_sim = cosine_similarity(menu_mat, menu_mat)
print(menu_sim.shape)
print('\n')

print(menu_sim[:2])
print('\n')


menu_sim_sorted_ind = menu_sim.argsort()[:, ::-1]
print(menu_sim_sorted_ind[:1])
print('\n')



def find_sim_cafe(df, sorted_ind, title_name, top_n=10):
    
    # 인자로 입력된 movies_df DataFrame에서 'title' 컬럼이 입력된 title_name 값인 DataFrame추출
    name_cafe = df[df['이름'] == title_name]
    
    # title_named을 가진 DataFrame의 index 객체를 ndarray로 반환하고 
    # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n 개의 index 추출
    title_index = name_cafe.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]
    
    # 추출된 top_n index들 출력. top_n index는 2차원 데이터 임. 
    #dataframe에서 index로 사용하기 위해서 1차원 array로 변경
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]



similar_cafe = find_sim_cafe(cafe_df, menu_sim_sorted_ind, '넉아웃',10)
similar_cafe[['이름', 'sentiment']]

################################################################################################
#########################################투표 보정식#############################################
###########################################################################################

cafe_df[['이름','sentiment','count']].sort_values('sentiment', ascending=False)[:10]


C = cafe_df['sentiment'].mean()
m = cafe_df['count'].quantile(0.6)
print('C:',round(C,3), 'm:',round(m,3))
print('\n')



percentile = 0.6
m = cafe_df['count'].quantile(percentile)
C = cafe_df['sentiment'].mean()

def weighted_vote_average(record):
    v = record['count']
    R = record['sentiment']
    
    return ( (v/(v+m)) * R ) + ( (m/(m+v)) * C )   

cafe_df['weighted_vote'] = cafe_df.apply(weighted_vote_average, axis=1)



cafe_df[['이름','sentiment','weighted_vote','count']].sort_values('weighted_vote', ascending=False)[:10]


def find_sim_cafe(df, sorted_ind, title_name, top_n=10):
    title_cafe = df[df['이름'] == title_name]
    title_index = title_cafe.index.values
    
    # top_n의 2배에 해당하는 쟝르 유사성이 높은 index 추출 
    similar_indexes = sorted_ind[title_index, :(top_n*2)]
    similar_indexes = similar_indexes.reshape(-1)
# 기준 영화 index는 제외
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    # top_n의 2배에 해당하는 후보군에서 weighted_vote 높은 순으로 top_n 만큼 추출 
    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]

print('\n')
similar_cafe = find_sim_cafe(cafe_df, menu_sim_sorted_ind, col_name,10)
similar_cafe[['이름', 'sentiment', 'weighted_vote']]
# print(menu_mat)